In [ ]:
!pip install telebot
!pip install mediapipe

import math
import telebot
import cv2
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt

In [ ]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

bot = telebot.TeleBot('7158807382:AAHlnxktPVVZ0oJQiWdgNkgTqrm0hg1RsLA')

pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)


@bot.message_handler(commands=['start'])
def main(message):
    text = 'Привет! Этот бот поможет вам обнаружить факт курения в неположенном месте.\nЧтобы продолжить, отправьте фото с человеком.'
    bot.send_message(message.chat.id, text)

@bot.message_handler(content_types=['photo', 'video'])
def handle_content(message):
    # Сохраняем полученное изображение или видео
    file_info = bot.get_file(message.photo[-1].file_id if message.content_type == 'photo' else message.video.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    with open("python1.png", "wb") as code:
        code.write(downloaded_file)
    image = cv2.imread("python1.png")
    output_image, landmarks = detectPose(image, pose, display=False)
    if landmarks:
        image, _ = classifyPose(landmarks, output_image, display=False)
    cv2.imwrite("image.png", image)
    photo = open("image.png", "rb")
    bot.send_photo(message.chat.id, photo)


def detectPose(image, pose, display=True):
    '''
    This function performs pose detection on an image.
    Args:
        image: The input image with a prominent person whose pose landmarks needs to be detected.
        pose: The pose setup function required to perform the pose detection.
        display: A boolean value that is if set to true the function displays the original input image, the resultant image,
                 and the pose landmarks in 3D plot and returns nothing.
    Returns:
        output_image: The input image with the detected pose landmarks drawn.
        landmarks: A list of detected landmarks converted into their original scale.
    '''
    # Create a copy of the input image.
    output_image = image.copy()

    # Convert the image from BGR into RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform the Pose Detection.
    results = pose.process(imageRGB)

    # Retrieve the height and width of the input image.
    height, width, _ = image.shape

    # Initialize a list to store the detected landmarks.
    landmarks = []

    # Check if any landmarks are detected.
    if results.pose_landmarks:

        # Draw Pose landmarks on the output image.
        mp_drawing.draw_landmarks(image=output_image, landmark_list=results.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS)

        # Iterate over the detected landmarks.
        for landmark in results.pose_landmarks.landmark:

            # Append the landmark into the list.
            landmarks.append((int(landmark.x * width), int(landmark.y * height),
                              (landmark.z * width)))

    # Check if the original input image and the resultant image are specified to be displayed.
    if display:

        # Display the original input image and the resultant image.
        plt.figure(figsize=[22 ,22])
        plt.subplot(121) \
        ;plt.imshow(image[: ,: ,::-1]) \
        ;plt.title("Original Image") \
        ;plt.axis('off');
        plt.subplot(122) \
        ;plt.imshow(output_image[: ,: ,::-1]) \
        ;plt.title("Output Image") \
        ;plt.axis('off');

        # Also Plot the Pose landmarks in 3D.
        mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)

    # Otherwise
    else:

        # Return the output image and the found landmarks.
        return output_image, landmarks

def calculateAngle(landmark1, landmark2, landmark3):
    '''
    This function calculates angle between three different landmarks.
    Args:
        landmark1: The first landmark containing the x,y and z coordinates.
        landmark2: The second landmark containing the x,y and z coordinates.
        landmark3: The third landmark containing the x,y and z coordinates.
    Returns:
        angle: The calculated angle between the three landmarks.

    '''

    # Get the required landmarks coordinates.
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    # Calculate the angle between the three points
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))

    # Check if the angle is less than zero.
    if angle < 0:
        angle = -angle

    # Return the calculated angle.
    return angle

def calculatedistance(landmark1, landmark2):
    # Get the required landmarks coordinates.
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    # Calculate the angle between the three points
    dist = math.sqrt((x2 -x1 )** 2 +(y2 -y1 )**2)
    return dist

def classifyPose(landmarks, output_image, display=False):
    '''
    This function classifies yoga poses depending upon the angles of various body joints.
    Args:
        landmarks: A list of detected landmarks of the person whose pose needs to be classified.
        output_image: A image of the person with the detected pose landmarks drawn.
        display: A boolean value that is if set to true the function displays the resultant image with the pose label
        written on it and returns nothing.
    Returns:
        output_image: The image with the detected pose landmarks drawn and pose label written.
        label: The classified pose label of the person in the output_image.

    '''

    # Initialize the label of the pose. It is not known at this stage.
    label = 'Good Boy'

    # Specify the color (Red) with which the label will be written on the image.
    color = (0, 255, 0)

    # Calculate the required angles.
    # ----------------------------------------------------------------------------------------------------------------

    # Get the angle between the left shoulder, elbow and wrist points.
    left_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])

    # Get the angle between the right shoulder, elbow and wrist points.
    right_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])

    # Get the distance between the left elbow, wrist  points.
    dist_left = calculatedistance(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value])
    dist_right = calculatedistance(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                   landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
    dist_left_wrist_nose = calculatedistance(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value],
                                             landmarks[mp_pose.PoseLandmark.NOSE.value])
    dist_right_wrist_nose = calculatedistance(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value],
                                              landmarks[mp_pose.PoseLandmark.NOSE.value])



    # ----------------------------------------------------------------------------------------------------------------

    # Check if it is the warrior II pose or the T pose.
    # As for both of them, both arms should be straight and shoulders should be at the specific angle.
    # ----------------------------------------------------------------------------------------------------------------

    # Check if the both arms are straight.
    if left_elbow_angle > 0 and left_elbow_angle < 45:
        # Check if distance between nose and wrist smaller than shoulder length.
        if dist_left > dist_left_wrist_nose:
            label = 'smoking'

    # ----------------------------------------------------------------------------------------------------------------

    # Check if it is the tree pose.
    # ----------------------------------------------------------------------------------------------------------------


    if right_elbow_angle > 0 and right_elbow_angle < 45:
        # Check if the other leg is bended at the required angle.
        if dist_right > dist_right_wrist_nose:
            # Specify the label of the pose that is tree pose.
            label = 'smoking'

    # ----------------------------------------------------------------------------------------------------------------

    # Check if the pose is classified successfully
    if label != 'Good Boy':

        # Update the color (to green) with which the label will be written on the image.
        color = (0, 0, 255)

    # Write the label on the output image.
    cv2.putText(output_image, label, (10, 30) ,cv2.FONT_HERSHEY_PLAIN, 2, color, 2)
    # Check if the resultant image is specified to be displayed.
    if display:

        # Display the resultant image.
        plt.figure(figsize=[10 ,10])
        plt.imshow(output_image[: ,: ,::-1]) \
        ;plt.title("Output Image") \
        ;plt.axis('off');

    else:

        # Return the output image and the classified label.
        return output_image, label


In [ ]:
    # Запуск бота
    bot.polling()